In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# Board Evaluation Functions

### Dependencies

In [ ]:
import chess
import chess.polyglot
import chess.gaviota

import import_ipynb
from Globals import Globals

<a id= 'EvaluateMove'> </a>
#### chess.Board.evaluate_move
Calculates the score of a move on a given board state. A high score represents a good move and a low score represents a bad move. The evaluation is performed over multiple phases, which account for game overs, endgame tablebase information, attacks (including en passant), positional changes, castling and promotions. Note that this score only reflects the move, not the overall state of the board. If the player making the move is in a disadvantageous position, but the chosen move makes the position less disadvantageous, the move score is positive.

This function is iterative, meaning that one can evaluate multiple moves back-to-back to retrieve a combined score of all moves in that sequence.

This implements the $\texttt{heuristic}$ function as described in [Introduction.ipynb](Introduction.ipynb#Heuristics). The parameters from this function are represented as follows:
* $\texttt{s}$: `self`, as the function is a member of the `chess.Board` class, which represents the chess board state.
* $\texttt{m}$: `move`.
* $\texttt{p}$: `self.turn`.
* $\texttt{o}$: `is_opponent_multiplier`, which is $-1$ if `is_opponent = True`, $1$ otherwise.

###### __<u>Arguments</u>__
``use_heuristic (bool):``  
Whether or not the heuristic for evaluating the chess board should be used. Chess problems don't need this heuristic. If the heuristic is disabled, only moves that result in a checkmate or draw are given a score.  

``is_opponent (bool):``  
Whether the move is made by the opponent (`True`) or not (`False`).

``prev_score (int):``  
The combined score of the previous evaluations in the search. The move's score is added to this value to calculate the total score.

``depth (int):``  
The remaining depth of the search, i.e. how many plies the search should still look into the future.  

``move (chess.Move):``  
The move that should be evaluated.

``endgame_tablebase (chess.gaviota.PythonTablebase):``  
The endgame tablebase attached to the game, which serves as a shortcut for ideal moves in the endgame.

###### __<u>Returns _(int, bool)_</u>__
- The total score (previous move scores + new move score).
- Whether or not the endgame tablebases were used to find the move.

In [ ]:
def evaluate_move(
    self,
    use_heuristic: bool,
    is_opponent: bool,
    prev_score: int,
    depth: int,
    move: chess.Move,
    endgame_tablebase: chess.gaviota.PythonTablebase
) -> (int, bool):

    color = self.turn
    is_opponent_multiplier = -1 if is_opponent else 1

    # Handle ending the game (draw = neutral, win = highest score, loss = lowest score)
    game_ended, winner = self.evaluate_game_end(move)
    if game_ended:
        if winner is None:
            draw_score = -Globals.EVALUATION_SCORE_DRAW if self.is_winning(color) \
                    else Globals.EVALUATION_SCORE_DRAW
            return draw_score * is_opponent_multiplier, False
        return (Globals.EVALUATION_SCORE_CHECKMATE + depth) * is_opponent_multiplier \
                * (1 if winner == color else -1), False

    # If the game hasn't ended and we don't use a chess board heuristic,
    # the score doesn't matter
    if not use_heuristic:
        return 0, False

    # Handle endgame tablebase
    if self.check_endgame_tablebase and endgame_tablebase is not None:
        endgame_tablebase_score = self.evaluate_endgame(move, endgame_tablebase)
        if endgame_tablebase_score is not None:
            return endgame_tablebase_score * is_opponent_multiplier, True

    # If the game hasn't ended and doesn't have a guaranteed win/draw/loss
    # from the endgame tablebase, evaluate the move normally
    score_diff = self.evaluate_attack(move)
    score_diff += self.evaluate_positions(color, move)
    score_diff += self.evaluate_castling(color, move)
    score_diff += self.evaluate_promotion(move)

    return prev_score + (score_diff * is_opponent_multiplier), False

chess.Board.evaluate_move = evaluate_move
del evaluate_move

#### chess.Board.evaluate_game_end
Finds if, after a given move, the chess game is over. If so, it also finds the winner of the game, or `None` if it's a draw.

###### __<u>Arguments</u>__
``move (chess.Move): ``  
The move that should be checked for a game over.

###### __<u>Returns _(bool, chess.Color)_</u>__
- Whether or not the move lead to a game over.
- The winning color if a game over occured, or `None` in case of a draw or if the game isn't over.

In [ ]:
def evaluate_game_end(self, move: chess.Move) -> (bool, chess.Color):
    self.push(move)
    game_ended = self.is_game_over()
    winner = None if self.outcome() is None else self.outcome().winner
    self.pop()

    return game_ended, winner

chess.Board.evaluate_game_end = evaluate_game_end
del evaluate_game_end

#### chess.Board.evaluate_endgame
Calculates a score based on the information in the endgame tablebases. This is based on the DTM (depth to mate) value, which lists the amount of half-moves until a forced mate. If this value is positive, the current side to move can win in that many half-moves. If the value is negative, the current side to move loses in that many half-moves (assuming perfect play by the opponent). If the value is zero, a draw is forced. The evaluation score is calculated as follows:
- If the DTM value is positive, the score is a very high positive number minus the DTM value (the fewer half-moves to win the better).
- If the DTM value is negative, the score is a very high negative number minus the DTM value (the more half-moves to lose the better, and since the DTM value is negative, the score is less negative for high DTM values).
- If the DTM value is zero, the score is positive if the player is in a disadvantageous position and negative if the player is in an advantageous position.

###### __<u>Arguments</u>__
``move (chess.Move):``  
The move that should be checked against the endgame tablebases.

``endgame_tablebase (chess.gaviota.PythonTablebase):``  
The endgame tablebase object attached to the game.

###### __<u>Returns _(int)_</u>__
The evaluation score if the endgame tablebases contained DTM information for the current board state (after the move is made), otherwise `None`.

In [ ]:
def evaluate_endgame(
    self,
    move: chess.Move,
    endgame_tablebase: chess.gaviota.PythonTablebase
) -> int:

    evaluation_score = 0
    self.push(move)

    try:
        dtm_score = -endgame_tablebase.probe_dtm(self) # Negate because self.push(move) switched turn
        if dtm_score > 0:
            evaluation_score = Globals.EVALUATION_SCORE_ENDGAME - dtm_score
        elif dtm_score < 0:
            evaluation_score = -Globals.EVALUATION_SCORE_ENDGAME - dtm_score
        else:
            evaluation_score = -Globals.EVALUATION_SCORE_DRAW if self.is_winning(not self.turn) \
                    else Globals.EVALUATION_SCORE_DRAW
    except KeyError:
        evaluation_score = None

    self.pop()
    return evaluation_score

chess.Board.evaluate_endgame = evaluate_endgame
del evaluate_endgame

#### chess.Board.evaluate_attack
Calculates the move score associated with an attack (capture), which is the piece value of the opponent's attacked piece. If the move is not an attack, this score is 0.

###### __<u>Arguments</u>__
``move (chess.Move):``  
The move that should be checked for an attack.

###### __<u>Returns _(int)_</u>__
The move score associated with the attack.

In [ ]:
def evaluate_attack(self, move: chess.Move) -> int:
    if self.is_en_passant(move):
        opposite_color = not self.piece_at(move.from_square).color
        to_piece = chess.Piece(chess.PAWN, opposite_color)
    else:
        to_piece = self.piece_at(move.to_square)
        
    # No need to check if the piece is of the opposite color.
    # Attacking one's own color is not a legal move.
    if to_piece is not None:
        return self.get_piece_value(to_piece.piece_type)
    return 0

chess.Board.evaluate_attack = evaluate_attack
del evaluate_attack

#### chess.Board.evaluate_positions
Calculates the move score associated with the "from" and "to" positions of a move. This is the "to" score minus the "from" score, each of which is retrieved from the piece square table of the corresponding piece.

###### __<u>Arguments</u>__
``color (chess.Color):``  
The color of the player who makes the move. 

``move (chess.Move):``  
The move of which the positions should be evaluated.

###### __<u>Returns _(int)_</u>__
The move score associated with the positions.

In [ ]:
def evaluate_positions(self, color: chess.Color, move: chess.Move) -> int:
    from_piece = self.piece_at(move.from_square)
    from_piece_square_table_key = self.get_piece_square_table_key(from_piece.piece_type)
    to_piece_type = from_piece.piece_type if move.promotion is None else move.promotion
    to_piece_square_table_key = self.get_piece_square_table_key(to_piece_type)

    from_score = self.get_positional_score(color,
            from_piece_square_table_key, move.from_square)
    to_score = self.get_positional_score(color,
            to_piece_square_table_key, move.to_square)
    return to_score - from_score

chess.Board.evaluate_positions = evaluate_positions
del evaluate_positions

#### chess.Board.get_positional_score
Finds the positional score of a piece, which is retrieved from the matching piece square table.

###### __<u>Arguments</u>__
``color (chess.Color):``  
The color of the piece for which the positional score should be found.

``piece_square_table_key (str):``  
The dictionary key to index the piece square table of the piece's type.

``square (chess.Square):``  
The square (position) that the piece is located at.

###### __<u>Returns _(int)_</u>__
The positional score.

In [ ]:
def get_positional_score(
    self, 
    color: chess.Color, 
    piece_square_table_key: str, 
    square: chess.Square
) -> int:

    hor_index = square % 8
    ver_index = 7 - square // 8 if color == chess.WHITE else square // 8
    piece_square_table = Globals.PIECE_SQUARE_TABLES[piece_square_table_key]
    piece_square_value = piece_square_table[ver_index][hor_index]
    return piece_square_value

chess.Board.get_positional_score = get_positional_score
del get_positional_score

#### chess.Board.evaluate_castling
Calculates the move score associated with castling. This score is the change in the positional score of the rook, as the king's positional score is already handled by `chess.Board.evaluate_positions`. If the move does not involve castling, this score is 0.

###### __<u>Arguments</u>__
``color (chess.Color):``   
The color of the player who makes the move.

``move (chess.Move):``  
The move for which castling should be evaluated.

###### __<u>Returns _(int)_</u>__
The move score associated with the castle.

In [ ]:
def evaluate_castling(self, color: chess.Color, move: chess.Move) -> int:
    if not self.is_castling(move):
        return 0

    old_square, new_square = self.find_castling_rook_squares(color, move)
    old_score = self.get_positional_score(color, 'r', old_square)
    new_score = self.get_positional_score(color, 'r', new_square)
    return new_score - old_score

chess.Board.evaluate_castling = evaluate_castling
del evaluate_castling

#### chess.Board.evaluate_promotion
Calculates the move score associated with a promotion, i.e. transforming a pawn into a queen, knight, rook or bishop. This score is defined as the piece value of the new piece minus the piece value of the pawn. If the move does not result in a promotion, this score is 0. The effect that a promotion has on the positional score is accounted for in `chess.Board.evaluate_positions`.

###### __<u>Arguments</u>__
``move (chess.Move):``  
The move for which a promotion should be evaluated.

###### __<u>Returns _(int)_</u>__
The move score associated with the promotion.

In [ ]:
def evaluate_promotion(self, move: chess.Move) -> int:
    if move.promotion is None:
        return 0

    old_piece = self.piece_at(move.from_square)
    old_piece_value = self.get_piece_value(old_piece.piece_type)
    new_piece_value = self.get_piece_value(move.promotion)
    return new_piece_value - old_piece_value

chess.Board.evaluate_promotion = evaluate_promotion
del evaluate_promotion